In [ ]:
!pip install requests pillow google-api-python-client beautifulsoup4 tqdm pexels_api python-dotenv selenium webdriver-manager
!pip install spacy webcolors jsonschema

!apt-get update
!apt-get install -y wget
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get install -y ./google-chrome-stable_current_amd64.deb

!pip install -q webdriver_manager

In [ ]:
# Selenium WebDriver Setup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.os_manager import ChromeType

# System & Environment
import os
import tempfile
import time
import random
import json
import zipfile
import copy
import logging
from pathlib import Path
from dotenv import load_dotenv
from pprint import pprint
import datetime
import re

# Image
from PIL import (
    Image,
    ImageDraw,
    ImageFont,
    ImageOps,
    ImageColor,
    UnidentifiedImageError
)
from io import BytesIO

# Web & API
import requests
from bs4 import BeautifulSoup
from googleapiclient.discovery import build
from pexels_api import API as PexelsAPI

# NLP & Parsing
import spacy
from spacy.matcher import Matcher

# Visualization
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import HTML
import webcolors
from scipy.spatial import KDTree
from colorsys import rgb_to_hls
from IPython.display import Image as ColabImage, display

# Colab
from google.colab import files

In [ ]:
def setup_colab_driver():
    logging.getLogger('WDM').setLevel(logging.NOTSET)

    # Chrome Options
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("--remote-debugging-port=9222")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("user-agent=Mozilla/5.0")

    # Binary path check
    chrome_path = "/usr/bin/google-chrome"
    if not os.path.exists(chrome_path):
        raise FileNotFoundError(f"Chrome binary not found at: {chrome_path}")
    options.binary_location = chrome_path

    # Temporary user data dir
    user_data_dir = tempfile.mkdtemp()
    options.add_argument(f"--user-data-dir={user_data_dir}")

    # Try WebDriver setup
    driver = None
    try:
        service = ChromeService(ChromeDriverManager(chrome_type=ChromeType.GOOGLE).install())
        driver = webdriver.Chrome(service=service, options=options)
        driver.get("https://www.google.com")
        print(f"WebDriver initialized. Page title: {driver.title}")
    except Exception as e:
        print(f"Error occurred: {e}")
        try:
            service = ChromeService(executable_path="/usr/bin/chromedriver")
            driver = webdriver.Chrome(service=service, options=options)
            driver.get("https://www.google.com")
            print(f"Fallback to system chromedriver. Page title: {driver.title}")
        except Exception as fallback_e:
            print(f"Fallback failed: {fallback_e}")

    return driver

In [ ]:
# Clear old assets
for f in os.listdir():
    if f.endswith((".json", ".png", ".ttf")):
        try:
            os.remove(f)
        except Exception as e:
            print(f"Could not remove {f}: {e}")
print("Cleared old assets. Remaining files:", os.listdir())

# Required files check
required_files = ["colour.json", "emotion_design_dataset.json", "Blank Template.png"]
missing_files = [f for f in required_files if not os.path.exists(f)]

if missing_files:
    print(" Missing files detected:")
    print("- " + "\n- ".join(missing_files))
    print("Please upload the following:")
    for _ in range(len(missing_files)):
        files.upload()
else:
    print("All required base files already present.")

# Load JSON and template image
with open("colour.json", "r") as f:
    color_data = json.load(f)
with open("emotion_design_dataset.json", "r") as f:
    emotion_dataset = json.load(f)

template_bg = Image.open("Blank Template.png").convert("RGB")
template_bg = template_bg.resize((1080, 1620))

# Prepare folders
web_ui_folder = "web_ui"
default_ui_folder = "default_ui"
os.makedirs(web_ui_folder, exist_ok=True)
os.makedirs(default_ui_folder, exist_ok=True)

# UI ZIP uploads only if folders are empty
zip_upload_needed = (
    not os.listdir(web_ui_folder) or
    not os.listdir(default_ui_folder)
)

if zip_upload_needed:
    print("\n UI folders are empty. Upload ZIP files:")
    print("- One for emotion-specific UI images")
    print("- One named 'Default.zip' for default UI assets")

    for _ in range(2):
        uploaded = files.upload()
        for filename in uploaded:
            with tempfile.TemporaryDirectory() as tmpdir:
                zip_path = os.path.join(tmpdir, filename)
                with open(zip_path, "wb") as f:
                    f.write(uploaded[filename])
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    target_folder = default_ui_folder if filename.lower().startswith("default") else web_ui_folder
                    zip_ref.extractall(target_folder)
                    print(f"Extracted {filename} to: {target_folder}")
else:
    print(" UI folders already populated. Skipping ZIP upload.")

In [ ]:
# Negation and Exclusion Keywords
NEGATION_WORDS = {"not", "no", "without", "except"}
NEGATION_PREFIXES = {"un", "non", "dis", "a"}
NEGATIVE_ADJ = {"unfriendly", "unpleasant", "ugly", "unattractive"}
EXCLUSION_VERBS = {"exclude", "remove", "omit", "avoid", "prevent"}

global nlp

# NLP Initialization
def initialize_nlp():
    global spacy
    try:
        return spacy.load("en_core_web_sm")
    except OSError:
        import spacy.cli
        spacy.cli.download("en_core_web_sm")
        return spacy.load("en_core_web_sm")

def is_negated(span):
    if isinstance(span, spacy.tokens.Doc): span = span[:]
    if any(token.dep_ == "neg" for token in span): return True
    if span.start > 0 and span.doc[span.start - 1].lower_ in NEGATION_WORDS: return True
    for token in span:
        if token.text.lower() in NEGATIVE_ADJ: return True
        if token.dep_ in {"dobj", "nsubjpass", "pobj"} and token.head.lemma_ in EXCLUSION_VERBS: return True
        for ancestor in token.ancestors:
            if ancestor.lemma_ in EXCLUSION_VERBS and token.dep_ in {"dobj", "nsubjpass", "pobj"}: return True
        for prefix in NEGATION_PREFIXES:
            if token.text.lower().startswith(prefix) and token.pos_ in {"ADJ", "NOUN"}: return True
    return False

def adjust_negation_contrast(color_candidates, doc, nlp):
    found, negated = set(), set()
    for phrase in color_candidates:
        span = nlp(phrase)
        if is_negated(span): negated.add(phrase)
        else: found.add(phrase)
    for i, token in enumerate(doc):
        if token.lower_ == "not":
            left_span = doc[max(i - 6, 0):i].text
            right_span = doc[i + 1:i + 7].text
            for phrase in color_candidates:
                if phrase in left_span: found.add(phrase)
                elif phrase in right_span: negated.add(phrase)
    unassigned = set(color_candidates) - found - negated
    found |= unassigned
    return sorted(found), sorted(negated)

def parse_prompt(prompt, nlp, matcher, color_keywords, pattern_keywords):
    doc = nlp(prompt.lower())
    explicit_found, explicit_negated, color_phrases = [], [], []

    for i in range(len(doc) - 1):
        if doc[i].pos_ == "ADJ" and doc[i + 1].text in color_keywords:
            phrase = f"{doc[i].text} {doc[i + 1].text}"
            color_phrases.append(phrase)

    matches = matcher(doc)
    for _, start, end in matches:
        span = doc[start:end]
        color_phrases.append(span.text.strip())

    for color_phrase in color_keywords:
        pattern = r'\b' + re.escape(color_phrase) + r'\b'
        for match in re.finditer(pattern, doc.text):
            span = doc.char_span(*match.span(), alignment_mode="expand")
            if span:
                if is_negated(span): explicit_negated.append(span.text)
                else: explicit_found.append(span.text)

    for phrase in color_phrases:
        if phrase not in explicit_found and phrase not in explicit_negated:
            explicit_found.append(phrase)

    all_candidates = list(set(explicit_found + explicit_negated))
    found_colors, negated_colors = adjust_negation_contrast(all_candidates, doc, nlp)

    visuals = []
    abstract_terms = {"mood", "emotion", "silence", "quietness", "gentle", "soft", "subtle", "neutral", "neutrals", "calm", "dark", "light"}
    for sent in doc.sents:
        parts = re.split(r"[,\-–—]", sent.text)
        for phrase in parts:
            phrase = phrase.strip()
            tokens = re.findall(r"\b\w+\b", phrase.lower())
            is_abstract = any(t in abstract_terms or t in color_keywords for t in tokens)
            if len(tokens) >= 2 and not is_abstract and not is_negated(nlp(phrase)):
                visuals.append(phrase)

    patterns = [kw for kw in pattern_keywords if re.search(r'\b' + re.escape(kw) + r'\b', doc.text) and not is_negated(nlp(kw))]

    return {
        "colors": sorted(set(found_colors)),
        "negated_colors": sorted(set(negated_colors)),
        "visuals": visuals,
        "patterns": patterns
    }

def is_prompt_generic(parsed):
    visuals = parsed.get("visuals", [])
    patterns = parsed.get("patterns", [])
    colors = parsed.get("colors", [])
    print(f"Parsed visuals: {visuals}")
    print(f"Parsed colors: {colors}")
    print(f"Parsed patterns: {patterns}")
    if not visuals and not patterns and not colors:
        return True
    if sum([len(visuals) >= 1, len(colors) >= 1, len(patterns) >= 1]) >= 2:
        return False
    if len(visuals) == 0 and len(colors) <= 1 and len(patterns) == 0:
        return True
    return False

def sanitize_emotion_label(label):
    label = label.strip().upper()
    return re.sub(r'[\s\-_\/]+', '/', label)

def merge_prompt_with_spec(parsed_data, emotion_dataset, label):
    base = copy.deepcopy(emotion_dataset.get(label, {}))
    if not base:
        print(f"Warning: Emotion label '{label}' not found in dataset. Using empty base.")
        base = { "colors": [], "images_keywords": [], "pattern": "" }

    parsed_colors = parsed_data.get("colors", [])
    if parsed_colors and isinstance(parsed_colors[0], dict):
        base["colors"] = parsed_colors

    abstract_terms = {
        "texture", "pattern", "emotion", "tone", "feeling", "expression",
        "mood", "moodboard", "quietness", "emptiness", "lighting", "shadow"
    }

    def is_visual_clean(phrase):
        tokens = re.findall(r"\b\w+\b", phrase.lower())
        return not any(t in abstract_terms for t in tokens)

    visuals_parsed = [v for v in parsed_data.get("visuals", []) if is_visual_clean(v)]
    visuals_json = [v for v in base.get("images_keywords", []) if is_visual_clean(v)]

    def get_base_tokens(phrase):
        return set(re.findall(r"\b\w+\b", phrase.lower()))

    selected_visuals = []
    candidate_pool = visuals_parsed + visuals_json
    random.shuffle(candidate_pool)

    for phrase in candidate_pool:
        current_tokens = get_base_tokens(phrase)
        if all(len(current_tokens.intersection(get_base_tokens(existing))) <= 1 for existing in selected_visuals):
            selected_visuals.append(phrase)
        if len(selected_visuals) >= 4:
            break

    base["all_images_keywords"] = candidate_pool
    base["images_keywords"] = selected_visuals

    patterns = parsed_data.get("patterns", [])
    if patterns:
        base["pattern"] = patterns[0]

    print(f"Merged spec: {json.dumps(base, indent=2)}")
    return base

def tweak_assets(spec, new_data):
    for key in ['colors', 'visuals', 'patterns']:
        if new_data.get(key):
            spec[key] = list(set(spec.get(key, [])) | set(new_data[key]))
    return spec

In [ ]:
def font(emotion_label, dataset_path="emotion_design_dataset.json", out_dir="/tmp/fonts"):
    FALLBACK_FONT = "https://fonts.gstatic.com/s/opensans/v18/mem8YaGs126MiZpBA-UFVZ0bf8pkAg.woff2"
    GOOGLE_FONTS_API_KEY = "Google Font API Key
    FONT_URL_OVERRIDES = {
        "Poppins": "https://fonts.gstatic.com/s/poppins/v20/pxiGyp8kv8JHgFVrJJfedw.woff2",
        "Open Sans": "https://fonts.gstatic.com/s/opensans/v18/mem8YaGs126MiZpBA-UFVZ0bf8pkAg.woff2"
    }

    def sanitize_font_filename(name):
        return "".join(c for c in name if c.isalnum()).lower()

    def sanitize_emotion_label(label):
        return re.sub(r'[\s\-_\/]+', '/', label.strip().upper())

    def build_fonts_service():
        return build("webfonts", "v1", developerKey=GOOGLE_FONTS_API_KEY)

    def fetch_font_url(font_name):
        if font_name in FONT_URL_OVERRIDES:
            return FONT_URL_OVERRIDES[font_name]
        try:
            service = build_fonts_service()
            fonts = service.webfonts().list().execute().get("items", [])
            for font in fonts:
                if font["family"].lower() == font_name.lower():
                    files = font.get("files", {})
                    return files.get("regular") or next((url for url in files.values() if ".ttf" in url.lower()), None)
        except Exception as e:
            print(f"Failed to fetch '{font_name}' from API: {e}")
        return None

    def download_font(name, url):
        slug = sanitize_font_filename(name)
        path = os.path.join(out_dir, f"{slug}.ttf")
        if os.path.exists(path):
            print(f"✔ Cached: {name} → {path}")
            return path
        try:
            os.makedirs(out_dir, exist_ok=True)
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            with open(path, "wb") as f:
                f.write(response.content)
            print(f"✔ Downloaded: {name} → {path}")
            return path
        except Exception as e:
            print(f"Download failed for '{name}': {e}")
            return FALLBACK_FONT

    def prepare(font_meta):
        font_paths = {}
        for role, meta in font_meta.items():
            font_name = meta.get("name")
            if not font_name:
                print(f"Missing font name for {role} — fallback used")
                font_paths[role] = FALLBACK_FONT
                continue
            print(f"Resolving font: {font_name} ({role})")
            font_url = fetch_font_url(font_name) or meta.get("url")
            font_paths[role] = download_font(font_name, font_url) if font_url else FALLBACK_FONT
        return font_paths

    # Main logic
    with open(dataset_path) as f:
        raw = json.load(f)
    dataset = {sanitize_emotion_label(k): v for k, v in raw.items()}
    label = sanitize_emotion_label(emotion_label)

    if label not in dataset:
        fallback = next((k for k in dataset if k.lower() == label.lower()), None)
        if fallback:
            label = fallback
            print(f"Using fallback label: {fallback}")
        else:
            print(f"Emotion label '{emotion_label}' not found")
            return {}

    font_meta = {}
    font_url_map = dataset[label].get("font_url", {})
    for role in ["headings", "body_text", "highlight_text"]:
        raw_label = dataset[label]["fonts"].get(role, "")
        font_name = raw_label.split(" (")[0].strip()
        if font_name:
            font_meta[role] = {
                "name": font_name,
                "url": font_url_map.get(font_name)
            }

    print(f"\nLoading fonts for: {label}")
    return prepare(font_meta)

In [ ]:
def image(merged_spec, emotion_label, api_key, colour_json_path="colour.json", save_dir="images"):
    def build_color_lookup(path):
        with open(path, "r") as f:
            col = json.load(f)
        name2rgb, vocab = {}, set()
        for group in col.values():
            for name in group:
                try:
                    name2rgb[name] = ImageColor.getrgb(name)
                except:
                    continue
        for name in name2rgb:
            c = name.lower()
            vocab.update([c, c + "s", c + "es"])
            if c.endswith("y"):
                vocab.add(c[:-1] + "ies")
        vocab.update([
            "greys", "grays", "pinks", "blacks", "whites", "nudes", "pastels", "earthtones",
            "reds", "blues", "yells", "beiges", "greens", "oranges"
        ])
        rgb_norm = [tuple(c / 255 for c in name2rgb[name]) for name in name2rgb]
        tree = KDTree(rgb_norm)
        return tree, list(name2rgb.keys()), name2rgb, vocab

    def is_safe_image(src: str, alt: str) -> bool:
        src, alt = src.lower(), alt.lower()
        unsafe_keywords = [
            "cover", "thumbnail", "video", "youtube", "vimeo", "play button",
            "logo", "svg", "ad", "template", "banner", "product", "shop", "sale",
            "explicit", "provocative", "sensual", "lingerie", "nude", "sex"
        ]
        return not any(bad in src or bad in alt for bad in unsafe_keywords)

    def save_image(url, path):
        if "s.pinimg.com/webapp" in url:
            return False
        try:
            response = requests.get(url, timeout=10, headers={"User-Agent": "Mozilla/5.0"})
            if "image" not in response.headers.get("Content-Type", ""):
                return False
            img = Image.open(BytesIO(response.content)).convert("RGB")
            img.save(path)
            return True
        except Exception as e:
            print(f"Failed to save image: {e}")
            return False

    def try_save_from_pool(urls, path):
        for url in urls:
            if save_image(url, path):
                return True
        print("No good image found in pool")
        return False

    def fetch_pinterest(phrase, color, count=8):
        options = Options()
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-gpu")
        options.add_argument("--remote-debugging-port=9222")
        options.binary_location = "/usr/bin/google-chrome"
        options.add_argument("--window-size=1920,1080")
        options.add_argument("user-agent=Mozilla/5.0")

        driver = None
        try:
            service = ChromeService(ChromeDriverManager(chrome_type=ChromeType.GOOGLE).install())
            driver = webdriver.Chrome(service=service, options=options)
        except Exception as e:
            print(f"WebDriverManager failed: {e}")
            try:
                service = ChromeService(executable_path="/usr/bin/chromedriver")
                driver = webdriver.Chrome(service=service, options=options)
            except Exception as fallback_e:
                print(f"System chromedriver fallback failed: {fallback_e}")
                return []

        emotion_tags = {
            "LOVE": ["romantic", "tender"], "JOY/HAPPINESS": ["bright", "cheerful"],
            "POWER/ANGER": ["bold", "fiery"], "PEACE/SERENITY": ["tranquil", "gentle"]
        }.get(emotion_label.upper(), [])

        query = '+'.join(filter(None, [phrase, color] + emotion_tags + [emotion_label.lower()]))
        search_url = f"https://www.pinterest.com/search/pins/?q={query}"

        try:
            driver.get(search_url)
            time.sleep(3)
            images = []
            for i in range(40):
                try:
                    pins = driver.find_elements(By.TAG_NAME, "img")
                    if i >= len(pins):
                        break
                    pin = pins[i]
                    src = pin.get_attribute("src")
                    alt = (pin.get_attribute("alt") or "")
                    if not src or "s.pinimg.com/webapp" in src:
                        continue
                    if not is_safe_image(src, alt):
                        continue
                    if any(c in f"{src} {alt}".lower() for c in color_vocab_all):
                        continue
                    images.append(src)
                    if len(images) >= count:
                        break
                except Exception as e:
                    print("Skipping stale image:", e)
                    continue
            return images
        except Exception as e:
            print(f"Error scraping Pinterest: {e}")
            return []
        finally:
            if driver:
                driver.quit()

    def fetch_pexels(phrase, color, count=8):
        client = PexelsAPI(api_key)
        client.search(f"{phrase} {color} atmosphere", page=1, results_per_page=count)
        raw_entries = client.get_entries()
        safe_urls = [p.original for p in raw_entries if is_safe_image(p.original, getattr(p, "alt", ""))]
        return safe_urls[:count]

    def fetch_images(phrase, color, count=8):
        results = fetch_pinterest(phrase, color, count)
        if len(results) < count:
            print("Pinterest low yield → fallback to Pexels")
            results += fetch_pexels(phrase, color, count - len(results))
        return results[:count]

    tree, color_names, name2rgb, color_vocab_all = build_color_lookup(colour_json_path)
    os.makedirs(save_dir, exist_ok=True)

    keywords = merged_spec.get("images_keywords", [])[:4]
    colors = merged_spec.get("colors", [])[:5]
    pattern = merged_spec.get("pattern", "")
    while len(keywords) < 4:
        keywords.append("texture shadow")
    while len(colors) < 5:
        colors.append({"hex": "#CCCCCC", "hue_group": "gray"})

    saved_paths = []

    for i, phrase in enumerate(keywords):
        color_name = colors[random.randint(0, 4)].get("hue_group", "gray")
        urls = fetch_images(phrase, color_name, count=8)
        path = os.path.join(save_dir, f"keyword_{i+1}.jpg")
        if try_save_from_pool(urls, path):
            saved_paths.append(path)

    pattern_color = colors[random.choice([1, 2, 3])].get("hue_group", "gray")
    urls = fetch_images(pattern, pattern_color, count=8)
    pattern_path = os.path.join(save_dir, "pattern.jpg")
    if try_save_from_pool(urls, pattern_path):
        saved_paths.append(pattern_path)
        merged_spec["pattern_path"] = pattern_path

    merged_spec["image_paths"] = saved_paths
    merged_spec["pattern_path"] = pattern_path
    return merged_spec

In [ ]:
def moodboard(spec, emotion_label, template_bg, emotion_dataset):
    # Ensure template is correct size
    if template_bg.size != (1080, 1620):
        template_bg = template_bg.resize((1080, 1620))

    FALLBACK_FONT_PATH = spec["font_paths"].get("headings")
    def clean_label(label):
        return re.sub(r"\W+", "", label.replace("/", "_").strip())

    def get_open_sans_font(size=18):
        try:
            path = spec["font_paths"].get("body_text", FALLBACK_FONT_PATH)
            return ImageFont.truetype(path, size=size)
        except Exception:
            return ImageFont.load_default()

    # Define zones for 1080x1620 output
    raw_zones = [
        (551, 496, 529, 601),
        (804, 1271, 276, 349),
        (361, 1271, 423, 349),
        (0, 1120, 341, 500),
        (0, 0, 535, 496),
        (551, 0, 529, 469)
    ]
    image_zones = [(x, y, x + w, y + h) for (x, y, w, h) in raw_zones]
    color_zones = [
        (360, 1120, 488, 1248),
        (508, 1120, 636, 1248),
        (656, 1120, 784, 1248),
        (804, 1120, 932, 1248),
        (952, 1120, 1080, 1248)
    ]
    font_block_zone = (0, 628, 530, 1097)  # Adjust if you need wider/narrower font area

    canvas = template_bg.copy()
    draw = ImageDraw.Draw(canvas)

    # Paste images
    def paste_and_crop(img_path, target_box, canvas):
        try:
            img = Image.open(img_path).convert("RGB")
            img = ImageOps.fit(
                img,
                (target_box[2] - target_box[0], target_box[3] - target_box[1]),
                Image.Resampling.LANCZOS
            )
            canvas.paste(img, (target_box[0], target_box[1]))
        except Exception as e:
            print(f"Failed to paste image '{img_path}': {e}")

    for i, path in enumerate(spec.get("image_paths", [])[:4]):
        paste_and_crop(path, image_zones[i], canvas)

    # Pattern
    pattern_path = spec.get("pattern_path")
    if pattern_path and os.path.exists(pattern_path):
        paste_and_crop(pattern_path, image_zones[4], canvas)

    # Website UI
    ui_filename = spec.get("website_ui", "")
    ui_path = os.path.join("web_ui", ui_filename)
    if ui_filename and os.path.isfile(ui_path):
        paste_and_crop(ui_path, image_zones[5], canvas)

    # Colors
    for zone, color_obj in zip(color_zones, spec.get("colors", [])):
        hex_code = color_obj["hex"] if isinstance(color_obj, dict) else color_obj
        try:
            rgb = tuple(int(hex_code.lstrip("#")[i:i+2], 16) for i in (0, 2, 4))
        except:
            rgb = (200, 200, 200)
        draw.rectangle(zone, fill=rgb)

    # Font block
    bg_rgb = tuple(int(spec["font_bg_color"].lstrip("#")[i:i+2], 16) for i in (0, 2, 4))
    fg_rgb = tuple(int(spec["font_color"].lstrip("#")[i:i+2], 16) for i in (0, 2, 4))
    draw.rectangle(font_block_zone, fill=bg_rgb)

    # Title Font
    main_title = emotion_label.upper()
    try:
        title_font_path = spec["font_paths"].get("headings", FALLBACK_FONT_PATH)
        main_font = ImageFont.truetype(title_font_path, size=55)
    except Exception:
        main_font = get_open_sans_font(size=55)
    title_bbox = draw.textbbox((0, 0), main_title, font=main_font)
    title_x = font_block_zone[0] + (font_block_zone[2] - font_block_zone[0]) // 2 - (title_bbox[2] - title_bbox[0]) // 2
    title_y = font_block_zone[1] - 110
    draw.text((title_x, title_y), main_title, fill="black", font=main_font)

    fonts = spec.get("fonts", {})
    font_paths = spec.get("font_paths", {})
    samples = emotion_dataset.get(emotion_label, {}).get("samples", {})

    try:
        label_font_path = font_paths.get("body_text", FALLBACK_FONT_PATH)
        label_font = ImageFont.truetype(label_font_path, size=18)
    except Exception:
        label_font = get_open_sans_font(size=18)

    y_cursor = font_block_zone[1] + 32
    max_y = font_block_zone[3] - 40

    for style in ["headings", "body_text", "highlight_text"]:
        descriptor = samples.get(style, "").strip()
        font_name = fonts.get(style, "Unnamed")
        font_path = font_paths.get(style, font_paths.get("headings"))
        if not font_path:
            continue
        try:
            sample_font = ImageFont.truetype(font_path, size=60)
        except Exception:
            sample_font = get_open_sans_font(size=60)

        label_line = f"{font_name} ({descriptor})" if descriptor else font_name
        label_bbox = draw.textbbox((0, 0), label_line, font=label_font)
        label_x = font_block_zone[0] + (font_block_zone[2] - font_block_zone[0]) // 2 - (label_bbox[2] - label_bbox[0]) // 2
        draw.text((label_x, y_cursor), label_line, fill=fg_rgb, font=label_font)
        y_cursor += label_bbox[3] - label_bbox[1] + 14

        emotion_key = emotion_label.split('/')[0].strip().capitalize()
        sample_line = f"Feel The {emotion_key}"
        sample_bbox = draw.textbbox((0, 0), sample_line, font=sample_font)
        sample_x = font_block_zone[0] + (font_block_zone[2] - font_block_zone[0]) // 2 - (sample_bbox[2] - sample_bbox[0]) // 2
        draw.text((sample_x, y_cursor), sample_line, fill=fg_rgb, font=sample_font)
        y_cursor += sample_bbox[3] - sample_bbox[1] + 30

        if y_cursor > max_y:
            break

    return canvas

In [ ]:
def apply_user_overrides(user_prompt: str, visual_spec: dict) -> dict:

    prompt = user_prompt.lower()

    # Replace image keyword logic
    img_replace = re.findall(r'replace\s+(\w+)\s+with\s+(\w+)', prompt)
    for old_img, new_img in img_replace:
        for key, value in visual_spec.items():
            if value == old_img:
                visual_spec[key] = new_img
                print(f"Replaced image '{old_img}' with '{new_img}' in '{key}'.")

    # Color override
    hex_match = re.search(r'use this colour\s+(#[0-9a-fA-F]{6})', prompt)
    if hex_match:
        visual_spec["accent_color"] = hex_match.group(1)
        print(f"Accent color overridden to '{hex_match.group(1)}'.")

    # Use specific image
    visual_match = re.findall(r'use this visual\s+(\w+)', prompt)
    for new_visual in visual_match:
        visual_spec["main_img"] = new_visual
        print(f"🔧 Main visual replaced with '{new_visual}'.")

    # Emotion or tone prompts
    tone_map = {
        "calm": "#A2D5F2", "warm": "#FFB366", "bold": "#FF3E3E", "serene": "#B9E3C6",
        "playful": "#FFC8E4", "dramatic": "#222222"
    }
    for tone in tone_map:
        if f"make it more {tone}" in prompt or f"add {tone} tone" in prompt:
            visual_spec["accent_color"] = tone_map[tone]
            print(f"Emotional tone set to '{tone}' → Color: {tone_map[tone]}")

    # Exclude element
    exclude_match = re.findall(r'remove (\w+)', prompt)
    for element in exclude_match:
        for key in list(visual_spec.keys()):
            if visual_spec[key] == element or key == element:
                visual_spec.pop(key)
                print(f"Removed element '{element}' from spec.")

    # Include directive
    include_match = re.findall(r'include (\w+)', prompt)
    for asset in include_match:
        visual_spec[f"custom_{asset}"] = asset
        print(f"Included '{asset}' as 'custom_{asset}' in spec.")

    return visual_spec


def run_override_prompt(spec, emotion_label, template_bg, emotion_dataset, session_dir, count):
    from IPython.display import clear_output
    import ipywidgets as widgets

    # UI Components
    override_toggle = widgets.ToggleButtons(
        options=['yes', 'no'],
        description='Specify overrides?',
        button_style='info'
    )

    directive_box = widgets.Text(
        placeholder='Type your override (e.g. replace X with Y)',
        description='Directive:',
        layout=widgets.Layout(width='100%')
    )

    apply_btn = widgets.Button(
        description='Apply Override',
        button_style='success',
        icon='check'
    )

    output_area = widgets.Output()

    # Apply Button Logic
    def on_apply_click(_):
        directive = directive_box.value.strip()
        with output_area:
            clear_output()
            if directive:
                updated = apply_user_overrides(directive, spec)
                canvas_bg = template_bg.copy()
                output_img = moodboard(updated, emotion_label, canvas_bg, emotion_dataset)
                output_path = os.path.join(session_dir, f"moodboard_{count}_override.png")
                output_img.save(output_path)
                print(f"Override moodboard saved to: {output_path}")
                display(ColabImage(filename=output_path, width=300))
            else:
                print("No override directive provided.")

    # Toggle Button Logic
    def on_toggle(change):
        clear_output(wait=True)
        display(override_toggle)
        output_area.clear_output()
        if change['new'] == 'yes':
            display(widgets.VBox([directive_box, apply_btn, output_area]))
        else:
            with output_area:
                print("No overrides applied.")

    apply_btn.on_click(on_apply_click)
    override_toggle.observe(on_toggle, names='value')

    display(override_toggle)

In [ ]:
def run(emotion_label, color_data, emotion_dataset, template_bg):

    api_key = "Pexel API Key
    # NLP Setup
    nlp = initialize_nlp()
    matcher = Matcher(nlp.vocab)
    matcher.add("IMPLICIT_COLOR_NOUN_COLOR", [[{"POS": "NOUN"}, {"LOWER": "color"}]])
    matcher.add("IMPLICIT_COLOR_COLOR_OF_NOUN", [[{"LOWER": "color"}, {"LOWER": "of"}, {"POS": "DET", "OP": "?"}, {"POS": "NOUN"}]])
    matcher.add("IMPLICIT_COLOR_SHADE_OF", [[{"LOWER": {"IN": ["shade", "tone", "hue", "tint"]}}, {"LOWER": "of"}, {"POS": "DET", "OP": "?"}, {"POS": "NOUN"}]])

    # Vocabulary Prep
    color_keywords = [c.lower() for group in color_data.values() for c in group]
    pattern_keywords = ["floral", "stripes", "polka dots", "abstract", "geometric", "waves"]

    sanitized_label = sanitize_emotion_label(emotion_label)
    session_dir = tempfile.mkdtemp()
    print(f"Session assets will be stored in: {session_dir}")

    font_paths = font(emotion_label)
    prompt = input("Enter a visual prompt: ").strip()

    parsed = parse_prompt(prompt, nlp, matcher, color_keywords, pattern_keywords)
    print("Parsed result:")
    print(json.dumps(parsed, indent=2))

    # Fallback Handling
    if is_prompt_generic(parsed):
        moodboard_file = emotion_dataset.get(sanitized_label, {}).get("moodboard")
        if moodboard_file:
            moodboard_path = os.path.join("default_ui", moodboard_file)
            if os.path.exists(moodboard_path):
                output_path = os.path.join(session_dir, "moodboard_default.png")
                Image.open(moodboard_path).save(output_path)
                print(f"Fallback moodboard saved to: {output_path}")
                display(ColabImage(filename=output_path, width=300))
            else:
                print(f"Fallback moodboard '{moodboard_file}' not found.")
        else:
            print(f"No fallback moodboard found for '{sanitized_label}'")
        return None

    # Moodboard Spec & Rendering
    spec = merge_prompt_with_spec(parsed, emotion_dataset, sanitized_label)
    spec["font_paths"] = font_paths
    spec = image(spec, emotion_label, api_key)

    canvas_bg = template_bg.copy()
    output_img = moodboard(spec, emotion_label, canvas_bg, emotion_dataset)
    output_path = os.path.join(session_dir, "moodboard.png")
    output_img.save(output_path)
    print(f"Moodboard saved to: {output_path}")
    display(ColabImage(filename=output_path, width=300))

    return spec, emotion_label, template_bg, emotion_dataset, session_dir

In [ ]:
_ = run(emotion_label, color_data, emotion_dataset, template_bg)

In [ ]:
spec_context = run(emotion_label, color_data, emotion_dataset, template_bg)

if spec_context:
    spec, emotion_label, template_bg, emotion_dataset, session_dir = spec_context
    run_override_prompt(spec, emotion_label, template_bg, emotion_dataset, session_dir, count=1)